<a href="https://colab.research.google.com/github/andrewgcodes/ConsensusTranscription/blob/main/ConsensusTranscriptionV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary installs

In [3]:
!pip install openai requests speechrecognition rapidfuzz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install pydub
!sudo apt update
!sudo apt install ffmpeg -y


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,058 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Pa

In [33]:
import os
import requests
import speech_recognition as sr
import openai
from pydub import AudioSegment

# OpenAI Whisper API settings
OPENAI_API_KEY = "YOUR API KEY"  # replace with your OpenAI API key
OPENAI_API_URL = "https://api.openai.com/v1/audio/transcriptions"
MODEL_ID = "whisper-1"
AUDIO_FILE_PATH = "audio.mp3"  # replace with your audio file path

# Deepgram API settings
DEEPGRAM_API_KEY = "YOUR API KEY"  # replace with your Deepgram API key
DEEPGRAM_API_URL = "https://api.deepgram.com/v1/listen"
LANGUAGE = "en"
MODEL = "general"
TIER = "nova"

# Configure OpenAI
openai.api_key = OPENAI_API_KEY

# Function for OpenAI Whisper API
def transcribe_openai():
    with open(AUDIO_FILE_PATH, "rb") as audio_file:
        response = openai.Audio.transcribe(MODEL_ID, audio_file)
    return response.text

# Function for Deepgram API
def transcribe_deepgram():
    headers = {
        "Authorization": "Token " + DEEPGRAM_API_KEY,
        "Content-Type": "audio/mpeg"  # Set this to the MIME type of your audio file
    }
    with open(AUDIO_FILE_PATH, "rb") as audio_file:
        audio_data = audio_file.read()
    response = requests.post(DEEPGRAM_API_URL, headers=headers, data=audio_data)
    response.raise_for_status()
    return response.json()["results"]["channels"][0]["alternatives"][0]["transcript"]


# Call the three transcription functions
openai_transcript = transcribe_openai()
deepgram_transcript = transcribe_deepgram()

# Print the results
print("OpenAI Transcript: ", openai_transcript)
print("Deepgram Transcript: ", deepgram_transcript)


OpenAI Transcript:  Tell you something if your friends start acting different when your nigga come around you need to cut that bitch off I will tell you to beat her ass right there and then but I don't want y'all motherfuckers saying that I'm promoting violence and shit Now, you know what the type of shit that I hate I hate when a nigga come around right and a bitch wanna change Her voice her tone and shit trying to talk all cute. You try to act all delegate talking about like Oh my god, are you serious bitch? Why are you talking like that bitch? Your voice is wild deep like fucking pop smoke all of a sudden you motherfucking talking like Kim Kardashian stop fucking playing Oh when she try to make you fucking look stupid like you ask a question like oh my god Remember that time that we almost drowned and she go like no, I don't remember. What are you talking about, bitch? Why you fucking trying to make me look fucking stupid bitch. I will whip your motherfucking ass I will ask her righ

In [34]:
from rapidfuzz import fuzz
import string
def remove_punctuation(input_string):
    # Make a translator object to remove all punctuation
    translator = str.maketrans('', '', string.punctuation)
    # Use the translator object to remove punctuation from the input string
    no_punct = input_string.translate(translator)
    return no_punct

def compare_strings(str1, str2):
    str1 = remove_punctuation(str1)
    str2 = remove_punctuation(str2)
    return fuzz.ratio(str1, str2)


similarity = compare_strings(openai_transcript.lower(), deepgram_transcript.lower())

print(f"The Levenshtein similarity between the two transcriptions is {similarity}%")


The Levenshtein similarity between the two transcriptions is 90.0462962962963%


In [35]:
def summarize_transcript():
    # Define the conversation
    messages = [
        {
            "role": "system",
            "content": "You are a editor, writer, and stenographer. Summarize the provided transcription text. Be aware that some words may be incorrect or missing. Do not censor expletives or curse words."
        },
        {
            "role": "user",
            "content": openai_transcript
        }
    ]

    # Request a completion from the API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=messages,
        max_tokens=100,
        temperature = 0
    )
    return response.choices[0].message["content"]

# Generate consensus transcription
audio_content = summarize_transcript()

In [38]:
# Request user input for the content description
# audio_content = input("Please describe the content of the audio file: ")

# Use OpenAI GPT-3.5-turbo to analyze the transcriptions
def analyze_transcriptions():
    # Define the conversation
    messages = [
        {
            "role": "system",
            "content": "You are a skilled editor, transcriber of speech, and stenographer. Your task is to review two transcripts of the same speech and write a new transcript. Given context that explains the speech, provide a new transcript that fixes the transcription errors in each of the two original transcripts. Make sure to consider how words that sound similar can be mistranscribed. Use your knowledge of phonetics, pronunciation, speech patterns, modern slang, and more. Generate a highly accurate consensus transcript that preserves the original words as exactly as possible while fixing the errors. Do not correct grammar based on Standard American English. Do not censor inappropriate words with asterisks. Think step by step and be careful to maintain accuracy to the transcripts when possible. Do not hallucinate. Keep the original words unless they are incorrect."
        },
        {
            "role": "user",
            "content": f"Here is a summary of the transcriptions:'{audio_content}'. Here are the two transcriptions which may have errors: OpenAI transcript: '{openai_transcript}', Deepgram transcript: '{deepgram_transcript}'. Provide a new corrected transcription that is faithful to the words used in the transcripts. Do not replace words with synonyms."
        }
    ]

    # Request a completion from the API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=messages, temperature =0
    )
    return response.choices[0].message["content"]

# Generate consensus transcription
consensus_transcript = analyze_transcriptions()

# Print the consensus transcription
print("Consensus Transcript: ", consensus_transcript)


Consensus Transcript:  Speaker: Tell you something, if your friends start acting different when your nigga come around, you need to cut that bitch off. I will tell you, beat her ass right there and then, but I don't want y'all motherfuckers saying that I'm promoting violence and shit. Now, you know what type of shit that I hate? I hate when a nigga come around, right, and a bitch wanna change her voice, her tone and shit, trying to talk all cute. You tryna act all delicate, talking about like, "Oh my god, are you serious, bitch?" Why are you talking like that, bitch? Your voice is wild deep like fucking Pop Smoke all of a sudden, you motherfucking talking like Kim Kardashian. Stop fucking playing. Or when she tryna make you fucking look stupid, like you ask a question like, "Oh my god, remember that time that we almost drowned?" And she go like, "No, I don't remember. What are you talking about, bitch?" Why you fucking trying to make me look fucking stupid, bitch? I will whip your moth